In [1]:
!pip install -q streamlit
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 36.3 MB/s eta 0:00:00
--2025-12-04 04:35:01--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64.deb [following]
--2025-12-04 04:35:01--  https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/8a32f7c6-649c-4f0d-806d-e14c19d0786d?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-04T05%3A24%3A47Z&rscd=attachment%3B+f

In [2]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import numpy as np
import json
from PIL import Image

# Page Config
st.set_page_config(
    page_title="Fish Species Classifier",
    page_icon="🐟",
    layout="centered",
    initial_sidebar_state="expanded"
)

# Sidebar
st.sidebar.title("🐟 Fish Classifier Settings")
st.sidebar.info("Upload any fish image to identify the species.")

# Load model
model = tf.keras.models.load_model("/content/best_fish_model_rebuild.keras")

# Load class names
with open("/content/class_names_rebuild (1).json", "r") as f:
    class_names = json.load(f)

# Main Title
st.markdown("<h1 style='text-align: center; color: #1E90FF;'>🐠 Fish Image Classification</h1>", unsafe_allow_html=True)
st.write("This model predicts the fish category using deep learning (MobileNetV2 / VGG16).")

# File Upload
uploaded_file = st.file_uploader("📷 Upload a fish image", type=["jpg", "jpeg", "png"])

if uploaded_file:
    # Show Uploaded Image
    image = Image.open(uploaded_file).convert("RGB")

    st.markdown("### 📌 Uploaded Image")
    st.image(image, caption="Preview", use_container_width=True)

    # Preprocessing
    img = image.resize((224, 224))
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, axis=0)

    # Prediction
    with st.spinner("🔍 Classifying... Please wait..."):
        preds = model.predict(img_array)[0]

    index = np.argmax(preds)
    predicted_label = class_names[index]
    confidence = preds[index] * 100

    # Output Section
    st.success(f"🎯 **Predicted Species:** {predicted_label}")

    # Confidence Bar
    st.write("### 🔥 Confidence Level")
    st.progress(int(confidence))

    st.info(f"Confidence Score: **{confidence:.2f}%**")

    # Show probabilities table
    st.write("### 📊 Prediction Probabilities")
    prob_dict = {class_names[i]: float(preds[i]) for i in range(len(class_names))}
    st.json(prob_dict)


Writing app.py


In [3]:
!streamlit run app.py --server.port 8501 --server.headless true &>/content/log.txt &


In [ ]:
!cloudflared tunnel --url http://localhost:8501 --no-autoupdate


2025-12-04T04:35:29Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-12-04T04:35:29Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-12-04T04:35:33Z INF +--------------------------------------------------------------------------------------------+
2025-12-04T04:35:33Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-12-04T04:35:33Z INF |  https://volt-interfaces-eventually-bring.trycloudflar